<a href="https://colab.research.google.com/github/MasterBeard/Clustering-Based-Labeling/blob/main/Implementation_Clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
import numpy.polynomial.polynomial as poly

# Define index tickers
index_tickers = {
    'SPX': '^GSPC',     # S&P 500
    'IXIC': '^IXIC',    # NASDAQ Composite
    'HSI': '^HSI',      # Hang Seng Index
    'DJI': '^DJI',      # Dow Jones Industrial Average
    'FCHI': '^FCHI',    # CAC 40
    'DAXI': '^GDAXI',   # DAX
    'N225': '^N225',    # Nikkei 225
    'KS11': '^KS11',    # KOSPI
    'SENSEX': '^BSESN', # BSE Sensex
    'STOXX50': '^STOXX50E'  # EURO STOXX 50
}

# Define date ranges
date_ranges = {
    'train': ("2005-01-01", "2015-01-01"),
    'val': ("2015-01-02", "2019-12-31"),
    'test': ("2020-01-01", "2024-10-31")
}

# Store matrices and labels for each data split
data_splits = {split: {'matrices_4x4': [], 'matrices_1st': [], 'labels3': []} for split in date_ranges}

# Window size
window_size = 11

# Retrieve and process data for each time range
for split, (start_date, end_date) in date_ranges.items():
    # Download index data within the specified time range
    index_data = {name: yf.download(ticker, start=start_date, end=end_date) for name, ticker in index_tickers.items()}

    # Create first-order and second-order derivative matrices for each index
    for index_name, data in index_data.items():
        # Extract required 'Open' and 'Close' data
        open_values = data['Open'].dropna().values
        close_values = data['Close'].dropna().values

        # Construct windowed matrices
        for start in range(len(data) - window_size + 1):
            # Extract each row of data
            open_row = open_values[start:start + window_size]
            close_row = close_values[start:start + window_size]

            # Normalize using the second last value in close_row
            normalization_factor = close_row[-2]
            open_row = open_row / normalization_factor
            close_row = close_row / normalization_factor

            # Create interleaved array
            combined = np.array([open_row[i // 2] if i % 2 == 0 else close_row[i // 2] for i in range(window_size * 2)])

            matrix_4x4 = combined[:-2].reshape(4, 5, 1)
            matrix_4x5 = combined[-10:].reshape(-1)

            data_splits[split]['matrices_4x4'].append(matrix_4x4)
            data_splits[split]['matrices_1st'].append(matrix_4x5)

            # Create label
            label3 = 1 if close_row[-1] > close_row[-2] else 0
            data_splits[split]['labels3'].append(label3)

# Convert each split's matrices into NumPy arrays
train_matrices_4x4 = np.array(data_splits['train']['matrices_4x4'])
train_matrices_4x5 = np.array(data_splits['train']['matrices_1st'])
train_labels3 = np.array(data_splits['train']['labels3'])

val_matrices_4x4 = np.array(data_splits['val']['matrices_4x4'])
val_matrices_4x5 = np.array(data_splits['val']['matrices_1st'])
val_labels3 = np.array(data_splits['val']['labels3'])

test_matrices_4x4 = np.array(data_splits['test']['matrices_4x4'])
test_matrices_4x5 = np.array(data_splits['test']['matrices_1st'])
test_labels3 = np.array(data_splits['test']['labels3'])

# Print the shape of each set to check the results
print(f"Train 4x4 matrices shape: {train_matrices_4x4.shape}")
print(f"Train 4x5 matrices shape: {train_matrices_4x5.shape}")
print(f"Train labels shape: {train_labels3.shape}")

print(f"Validation 4x4 matrices shape: {val_matrices_4x4.shape}")
print(f"Validation 4x5 matrices shape: {val_matrices_4x5.shape}")
print(f"Validation labels shape: {val_labels3.shape}")

print(f"Test 4x4 matrices shape: {test_matrices_4x4.shape}")
print(f"Test 4x5 matrices shape: {test_matrices_4x5.shape}")
print(f"Test labels shape: {test_labels3.shape}")

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

Train 4x4 matrices shape: (24336, 4, 5, 1)
Train 4x5 matrices shape: (24336, 10)
Train labels shape: (24336,)
Validation 4x4 matrices shape: (12363, 4, 5, 1)
Validation 4x5 matrices shape: (12363, 10)
Validation labels shape: (12363,)
Test 4x4 matrices shape: (11993, 4, 5, 1)
Test 4x5 matrices shape: (11993, 10)
Test labels shape: (11993,)


In [2]:
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

# Set the number of clusters (can be adjusted as needed)
n_clusters = 15

# Initialize the KMeans model and fit it on the training set
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
train_clusters = kmeans.fit_predict(train_matrices_4x5)

# Predict clusters for the validation and test sets
val_clusters = kmeans.predict(val_matrices_4x5)
test_clusters = kmeans.predict(test_matrices_4x5)

# Get cluster centers
cluster_centers = kmeans.cluster_centers_

# Sort the cluster centers based on the last value of each center
sorted_indices = np.argsort(cluster_centers[:, -1])  # Get the sorted indices
sorted_cluster_centers = cluster_centers[sorted_indices]  # Reorder the centers using the sorted indices

# Print the sorted cluster centers and their corresponding cluster IDs
print("Sorted Cluster Centers (Centroids) based on the last value:")
for idx, center in zip(sorted_indices, sorted_cluster_centers):
    print(f"Cluster {idx}: {center}")

# Check the sample count distribution across clusters
print("\nTrain Cluster Distribution:")
unique_train, counts_train = np.unique(train_clusters, return_counts=True)
for cluster, count in zip(unique_train, counts_train):
    print(f"Cluster {cluster}: {count} samples")

print("\nValidation Cluster Distribution:")
unique_val, counts_val = np.unique(val_clusters, return_counts=True)
for cluster, count in zip(unique_val, counts_val):
    print(f"Cluster {cluster}: {count} samples")

print("\nTest Cluster Distribution:")
unique_test, counts_test = np.unique(test_clusters, return_counts=True)
for cluster, count in zip(unique_test, counts_test):
    print(f"Cluster {cluster}: {count} samples")

Sorted Cluster Centers (Centroids) based on the last value:
Cluster 13: [1.08971399 1.08110617 1.07636711 1.05882452 1.05122376 1.02779489
 1.01732592 1.         0.99164039 0.97368976]
Cluster 1: [0.99174393 0.99285541 0.99411333 0.99669753 0.9978181  0.9996557
 1.00022922 1.         0.99538042 0.9838447 ]
Cluster 10: [1.01671224 1.01374226 1.01191719 1.00502276 1.00322311 0.99936707
 0.99935775 1.         0.9958884  0.98514452]
Cluster 4: [0.92983071 0.9222671  0.92447353 0.93142844 0.9381301  0.95989943
 0.97171763 1.         1.00025388 0.99729041]
Cluster 5: [1.01362832 1.01466222 1.01610179 1.01884694 1.01894349 1.01597453
 1.01302796 1.         0.99968528 0.99857935]
Cluster 0: [1.04020315 1.03856483 1.03808774 1.03498569 1.03309469 1.02362124
 1.01821131 1.         1.00021898 0.99918126]
Cluster 12: [0.95428166 0.95455303 0.95737509 0.96515371 0.97026989 0.98235663
 0.98786494 1.         0.99989673 0.9998512 ]
Cluster 8: [0.97249402 0.97466675 0.97666833 0.98155529 0.98448479 0.9

In [3]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np

def make_model():
    """
    Create a convolutional neural network model with adjusted parameters.
    Parameter configuration:
    - conv1_filters: 64
    - conv2_filters: 128
    - dense_units: 256
    - dropout_rate: 0.1
    """
    model = tf.keras.Sequential([
        # First convolutional layer - adjusted to 64 filters
        layers.Conv2D(64, (2, 2), activation='relu', input_shape=(4, 5, 1)),
        layers.MaxPooling2D((2, 2), padding='same'),

        # Second convolutional layer - adjusted to 128 filters
        layers.Conv2D(128, (2, 2), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2), padding='same'),

        # Flatten layer
        layers.Flatten(),

        # Fully connected layer - adjusted to 256 units
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.1),  # Adjusted to 10% dropout

        # Output layer
        layers.Dense(15, activation='softmax')  # Output 15 class probabilities
    ])

    model.compile(
        optimizer='adam',
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
        metrics=['accuracy']
    )
    return model


# Train on the first data set (based on first-order derivatives)
print("Training on first derivative matrices with adjusted parameters...")
model1 = make_model()
history1 = model1.fit(
    train_matrices_4x4, train_clusters,
    epochs=300,
    batch_size=64,  # Adjusted to 64
    validation_data=(val_matrices_4x4, val_clusters),
    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)],
    verbose=1
)

# Evaluate the first model on the test set
test_loss1, test_accuracy1 = model1.evaluate(test_matrices_4x4, test_clusters)
print(f"Adjusted Model - Test Loss: {test_loss1:.4f}, Test Accuracy: {test_accuracy1:.4f}")

# Save model 1
model1.save('modelcnn_adjusted.h5')

Training on first derivative matrices with adjusted parameters...
Epoch 1/300


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


381/381 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.1852 - loss: 2.4049 - val_accuracy: 0.2987 - val_loss: 2.1547
Epoch 2/300
381/381 ━━━━━━━━━━━━━━━━━━━━ 6s 15ms/step - accuracy: 0.1945 - loss: 2.3365 - val_accuracy: 0.2976 - val_loss: 2.0996
Epoch 3/300
381/381 ━━━━━━━━━━━━━━━━━━━━ 8s 8ms/step - accuracy: 0.2504 - loss: 2.1246 - val_accuracy: 0.4601 - val_loss: 1.6058
Epoch 4/300
381/381 ━━━━━━━━━━━━━━━━━━━━ 6s 11ms/step - accuracy: 0.3948 - loss: 1.6691 - val_accuracy: 0.5267 - val_loss: 1.3940
Epoch 5/300
381/381 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.4325 - loss: 1.5414 - val_accuracy: 0.5225 - val_loss: 1.3468
Epoch 6/300
381/381 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.4496 - loss: 1.4689 - val_accuracy: 0.5327 - val_loss: 1.2839
Epoch 7/300
381/381 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.4671 - loss: 1.4297 - val_accuracy: 0.5558 - val_loss: 1.2412
Epoch 8/300
381/381 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.4833 - loss: 1.3800 - val_accuracy: 0.5

Adjusted Model - Test Loss: 0.5944, Test Accuracy: 0.7759


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import zipfile
import os
import pandas as pd
import numpy as np
import numpy.polynomial.polynomial as poly

# Set file path
zip_file_path = '/content/drive/My Drive/SP500_data2020-2024.zip'
output_dir = '/content/SP500_data2020-2024'

# Extract the zip file
os.makedirs(output_dir, exist_ok=True)
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(output_dir)

print("Files extracted to:", output_dir)
csv_files = sorted([f for f in os.listdir(output_dir) if f.endswith('.csv')])
print(f"Found {len(csv_files)} CSV files.")
print("Sample files:", csv_files[:5])

# Read all CSV files into a dictionary
sp500_data = {}
for csv_file in csv_files:
    symbol = csv_file.replace('.csv', '')
    file_path = os.path.join(output_dir, csv_file)
    sp500_data[symbol] = pd.read_csv(file_path)

# Filter out symbols with the most common data length
def filter_symbols_by_most_common_length(sp500_data):
    lengths = {symbol: len(data) for symbol, data in sp500_data.items()}
    length_counts = pd.Series(lengths).value_counts()
    most_common_length = length_counts.idxmax()
    print(f"Most common length: {most_common_length}, Count: {length_counts.max()}")
    filtered_symbols = [symbol for symbol, length in lengths.items() if length == most_common_length]
    filtered_data = {symbol: sp500_data[symbol] for symbol in filtered_symbols}
    return filtered_data, most_common_length

filtered_sp500_data, most_common_length = filter_symbols_by_most_common_length(sp500_data)
print(f"Filtered data count: {len(filtered_sp500_data)}")

# Parameters
window_size = 11

# Output data
matrices_4x4 = []
labels3 = []

# Generate matrices and labels
for symbol, data in filtered_sp500_data.items():
    data['Open'] = pd.to_numeric(data['Open'], errors='coerce')
    data['Close'] = pd.to_numeric(data['Close'], errors='coerce')
    open_values = data['Open'].values[2:]
    close_values = data['Close'].values[2:]

    num_samples = len(data) - window_size + 1
    for start in range(num_samples - 2):
        open_row = open_values[start:start + window_size]
        close_row = close_values[start:start + window_size]
        normalization_factor = close_row[-2]
        open_row = open_row / normalization_factor
        close_row = close_row / normalization_factor

        combined = np.array([open_row[i // 2] if i % 2 == 0 else close_row[i // 2] for i in range(window_size * 2)])
        matrix_4x4 = combined[:-2].reshape(4, 5, 1)
        label3 = 1 if close_row[-1] > close_row[-2] else 0

        matrices_4x4.append(matrix_4x4)
        labels3.append(label3)

# Convert to NumPy arrays
matrices_4x4 = np.array(matrices_4x4)
labels3 = np.array(labels3)

# Output shapes
print(f"4x4 matrices shape: {matrices_4x4.shape}")
print(f"Labels3 shape: {labels3.shape}")

Files extracted to: /content/SP500_data2020-2024
Found 501 CSV files.
Sample files: ['A.csv', 'AAPL.csv', 'ABBV.csv', 'ABNB.csv', 'ABT.csv']
Most common length: 1239, Count: 489
Filtered data count: 489
4x4 matrices shape: (600003, 4, 5, 1)
Labels3 shape: (600003,)


In [10]:
predictionscnn_pro = model1.predict(matrices_4x4)

# predict
predicted_classes = np.argmax(predictionscnn_pro, axis=1)

18751/18751 ━━━━━━━━━━━━━━━━━━━━ 43s 2ms/step


In [11]:
import numpy as np
from sklearn.metrics import confusion_matrix

# Get the total number of classes
num_classes = predictionscnn_pro.shape[1]

# Initialize the final prediction array
final_predictions = np.zeros(predictionscnn_pro.shape[0])  # Initialize to 0

# Define which class(es) should be marked as 1
target_classes = {sorted_indices[-1]}  # Assume sorted_indices is predefined

# Iterate over each sample
for i in range(predictionscnn_pro.shape[0]):
    # Get the predicted class for the current sample
    class_label = predicted_classes[i]

    # If the class is in the target set, label as 1; otherwise, 0
    final_predictions[i] = 1 if class_label in target_classes else 0

# Compute accuracy
accuracy = np.mean(final_predictions == labels3)

# Create the confusion matrix
cm = confusion_matrix(labels3, final_predictions, labels=[0, 1])

# Compute Precision
tp = cm[1, 1]  # True Positives
fp = cm[0, 1]  # False Positives
precision = tp / (tp + fp) if (tp + fp) != 0 else 0

# Compute Recall
fn = cm[1, 0]  # False Negatives
recall = tp / (tp + fn) if (tp + fn) != 0 else 0

# Compute F1 Score
f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0

# Print results
print("Final Evaluation Results:")
print(f"Accuracy: {accuracy:.2%}")
print(f"Precision (TP / (TP + FP)): {precision:.2%}")
print(f"Recall (TP / (TP + FN)): {recall:.2%}")
print(f"F1 Score: {f1:.2%}")
print(f"Confusion Matrix:\n{cm}")

Final Evaluation Results:
Accuracy: 48.39%
Precision (TP / (TP + FP)): 61.54%
Recall (TP / (TP + FN)): 0.82%
F1 Score: 1.62%
Confusion Matrix:
[[287794   1597]
 [308057   2555]]


In [12]:
import tensorflow as tf
from tensorflow import keras

def make_model():
    """
    Create an MLP model using the best parameter configuration:
    - hidden_units1: 128
    - hidden_units2: 128
    - dropout_rate: 0.0
    - learning_rate: 0.001
    """
    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(4, 5, 1)),  # Flatten input from (4, 5, 1) to (20,)
        tf.keras.layers.Dense(units=128, activation='relu'),  # First hidden layer with 128 units
        tf.keras.layers.Dense(units=128, activation='relu'),  # Second hidden layer with 128 units
        # Note: dropout_rate = 0.0 means no Dropout layer used
        tf.keras.layers.Dense(units=15, activation='softmax')  # Output layer with 15 classes
    ])

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

# Create and train the model
model1 = make_model()
history1 = model1.fit(
    train_matrices_4x4, train_clusters,
    epochs=300,
    batch_size=64,  # Using best batch_size = 64
    validation_data=(val_matrices_4x4, val_clusters),
    callbacks=[tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=20,
        restore_best_weights=True)],
    verbose=1
)

# Evaluate the model on the test set
test_loss1, test_accuracy1 = model1.evaluate(test_matrices_4x4, test_clusters)
print(f"Optimized MLP Model - Test Loss: {test_loss1:.4f}, Test Accuracy: {test_accuracy1:.4f}")

# Save the model
model1.save('optimized_mlp_model.h5')

Epoch 1/300


/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


381/381 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - accuracy: 0.1844 - loss: 2.3891 - val_accuracy: 0.2104 - val_loss: 2.1767
Epoch 2/300
381/381 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.1941 - loss: 2.3370 - val_accuracy: 0.2104 - val_loss: 2.1225
Epoch 3/300
381/381 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.2038 - loss: 2.2914 - val_accuracy: 0.3092 - val_loss: 2.0367
Epoch 4/300
381/381 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.2378 - loss: 2.1770 - val_accuracy: 0.3339 - val_loss: 1.9884
Epoch 5/300
381/381 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.2607 - loss: 2.0850 - val_accuracy: 0.3902 - val_loss: 1.8509
Epoch 6/300
381/381 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.2916 - loss: 1.9893 - val_accuracy: 0.4303 - val_loss: 1.8005
Epoch 7/300
381/381 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.3185 - loss: 1.8841 - val_accuracy: 0.4248 - val_loss: 1.6927
Epoch 8/300
381/381 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.3561 - loss: 1.7875 - val_accuracy: 0.401

Optimized MLP Model - Test Loss: 0.5611, Test Accuracy: 0.7886


In [13]:
# Get model prediction probabilities
predictionscnn_pro = model1.predict(matrices_4x4)

# Get predicted class labels
predicted_classes = np.argmax(predictionscnn_pro, axis=1)

18751/18751 ━━━━━━━━━━━━━━━━━━━━ 28s 1ms/step


In [14]:
import numpy as np
from sklearn.metrics import confusion_matrix

# Get the total number of classes
num_classes = predictionscnn_pro.shape[1]

# Initialize the final predictions array
final_predictions = np.zeros(predictionscnn_pro.shape[0])  # Initialize with 0

# Define the target class(es) to be labeled as 1
target_classes = {sorted_indices[-1]}  # Assume sorted_indices is predefined

# Iterate through each sample
for i in range(predictionscnn_pro.shape[0]):
    # Get the predicted class label for the current sample
    class_label = predicted_classes[i]

    # If the class is in the target set, assign 1; otherwise, 0
    final_predictions[i] = 1 if class_label in target_classes else 0

# Calculate accuracy
accuracy = np.mean(final_predictions == labels3)

# Create confusion matrix
cm = confusion_matrix(labels3, final_predictions, labels=[0, 1])

# Calculate Precision
tp = cm[1, 1]  # True Positives
fp = cm[0, 1]  # False Positives
precision = tp / (tp + fp) if (tp + fp) != 0 else 0

# Calculate Recall
fn = cm[1, 0]  # False Negatives
recall = tp / (tp + fn) if (tp + fn) != 0 else 0

# Calculate F1 Score
f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0

# Print results
print("Final Evaluation Results:")
print(f"Accuracy: {accuracy:.2%}")
print(f"Precision (TP / (TP + FP)): {precision:.2%}")
print(f"Recall (TP / (TP + FN)): {recall:.2%}")
print(f"F1 Score: {f1:.2%}")
print(f"Confusion Matrix:\n{cm}")

Final Evaluation Results:
Accuracy: 48.42%
Precision (TP / (TP + FP)): 59.40%
Recall (TP / (TP + FN)): 1.13%
F1 Score: 2.22%
Confusion Matrix:
[[286991   2400]
 [307100   3512]]


In [15]:
import tensorflow as tf
from tensorflow import keras

def build_lstm_model():
    """
    Build an LSTM model with adjusted parameters.
    Configuration:
    - input_shape: (20, 1)  # 20 time steps, 1 feature per step
    - lstm_units: 64
    - num_layers: 2 (two LSTM layers)
    - dense_units: 15
    - dropout_rate: 0.0
    - learning_rate: 0.0005
    """
    model = tf.keras.models.Sequential([
        keras.layers.Input(shape=(20, 1)),  # Set input to 20 time steps

        # First LSTM layer - 64 units, return sequences for next layer
        keras.layers.LSTM(64, return_sequences=True),

        # Second LSTM layer - 64 units, do not return sequences
        keras.layers.LSTM(64, return_sequences=False),

        # Output layer - 15 units
        keras.layers.Dense(15, activation='softmax')
    ])

    # Use Adam optimizer with adjusted learning rate of 0.0005
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.0005)

    model.compile(
        optimizer=optimizer,
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

# Build the model
lstm_model = build_lstm_model()

# Reshape training data to (samples, 20, 1)
train_data = train_matrices_4x4.reshape(-1, 20, 1)
val_data = val_matrices_4x4.reshape(-1, 20, 1)
test_data = test_matrices_4x4.reshape(-1, 20, 1)

# Train the model
history_lstm = lstm_model.fit(
    train_data, train_clusters,
    epochs=300,
    batch_size=64,
    validation_data=(val_data, val_clusters),
    callbacks=[tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=20,
        restore_best_weights=True)],
    verbose=1
)

# Evaluate the model on the test set
test_loss_lstm, test_accuracy_lstm = lstm_model.evaluate(
    test_data, test_clusters
)

print(f"Optimized LSTM Model (20 timesteps) - Test Loss: {test_loss_lstm:.4f}, Test Accuracy: {test_accuracy_lstm:.4f}")

# Save the model
lstm_model.save('optimized_lstm_model.h5')

Epoch 1/300
381/381 ━━━━━━━━━━━━━━━━━━━━ 20s 40ms/step - accuracy: 0.1921 - loss: 2.4044 - val_accuracy: 0.2104 - val_loss: 2.1821
Epoch 2/300
381/381 ━━━━━━━━━━━━━━━━━━━━ 15s 40ms/step - accuracy: 0.1931 - loss: 2.3469 - val_accuracy: 0.3048 - val_loss: 2.1593
Epoch 3/300
381/381 ━━━━━━━━━━━━━━━━━━━━ 21s 43ms/step - accuracy: 0.2183 - loss: 2.2520 - val_accuracy: 0.3955 - val_loss: 1.8175
Epoch 4/300
381/381 ━━━━━━━━━━━━━━━━━━━━ 18s 37ms/step - accuracy: 0.2762 - loss: 2.0068 - val_accuracy: 0.3475 - val_loss: 1.8234
Epoch 5/300
381/381 ━━━━━━━━━━━━━━━━━━━━ 23s 43ms/step - accuracy: 0.3066 - loss: 1.9012 - val_accuracy: 0.4683 - val_loss: 1.5626
Epoch 6/300
381/381 ━━━━━━━━━━━━━━━━━━━━ 18s 38ms/step - accuracy: 0.3599 - loss: 1.7434 - val_accuracy: 0.4226 - val_loss: 1.5926
Epoch 7/300
381/381 ━━━━━━━━━━━━━━━━━━━━ 20s 35ms/step - accuracy: 0.3709 - loss: 1.7141 - val_accuracy: 0.4761 - val_loss: 1.4526
Epoch 8/300
381/381 ━━━━━━━━━━━━━━━━━━━━ 22s 39ms/step - accuracy: 0.4138 - loss: 1

Optimized LSTM Model (20 timesteps) - Test Loss: 0.5307, Test Accuracy: 0.7848


In [16]:
# Get model prediction probabilities
predictionscnn_pro = lstm_model.predict(matrices_4x4.reshape(-1, 20, 1))

# Get predicted class labels
predicted_classes = np.argmax(predictionscnn_pro, axis=1)

18751/18751 ━━━━━━━━━━━━━━━━━━━━ 161s 9ms/step


In [17]:
import numpy as np
from sklearn.metrics import confusion_matrix

# Get the total number of classes
num_classes = predictionscnn_pro.shape[1]

# Initialize the final predictions array
final_predictions = np.zeros(predictionscnn_pro.shape[0])  # Initialize to 0

# Define the class(es) to be labeled as 1
target_classes = {sorted_indices[-1]}  # Assume sorted_indices is predefined

# Loop through each sample
for i in range(predictionscnn_pro.shape[0]):
    # Get the predicted class label for the current sample
    class_label = predicted_classes[i]

    # If the predicted class is in the target class set, mark as 1; otherwise, 0
    final_predictions[i] = 1 if class_label in target_classes else 0

# Calculate accuracy
accuracy = np.mean(final_predictions == labels3)

# Create the confusion matrix
cm = confusion_matrix(labels3, final_predictions, labels=[0, 1])

# Calculate Precision
tp = cm[1, 1]  # True Positives
fp = cm[0, 1]  # False Positives
precision = tp / (tp + fp) if (tp + fp) != 0 else 0

# Calculate Recall
fn = cm[1, 0]  # False Negatives
recall = tp / (tp + fn) if (tp + fn) != 0 else 0

# Calculate F1 Score
f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0

# Print results
print("Final Evaluation Results:")
print(f"Accuracy: {accuracy:.2%}")
print(f"Precision (TP / (TP + FP)): {precision:.2%}")
print(f"Recall (TP / (TP + FN)): {recall:.2%}")
print(f"F1 Score: {f1:.2%}")
print(f"Confusion Matrix:\n{cm}")

Final Evaluation Results:
Accuracy: 48.40%
Precision (TP / (TP + FP)): 60.58%
Recall (TP / (TP + FN)): 0.94%
F1 Score: 1.84%
Confusion Matrix:
[[287501   1890]
 [307707   2905]]
